In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../datasets/ECGdataset(Ass4)/ecg_autoencoder_dataset.csv")

In [3]:
x = data.values
x

array([[-1.1008778 , -3.9968398 , -4.2858426 , ...,  1.1196209 ,
        -1.4362499 ,  1.        ],
       [-0.56708802, -2.5934502 , -3.8742297 , ...,  0.90422673,
        -0.42179659,  1.        ],
       [ 0.49047253, -1.9144071 , -3.6163638 , ...,  1.403011  ,
        -0.38356426,  1.        ],
       ...,
       [-1.1229693 , -2.2529248 , -2.8676281 , ..., -2.0083694 ,
        -1.8083338 ,  0.        ],
       [-0.54770461, -1.8895451 , -2.8397786 , ...,  1.1504486 ,
         0.80493225,  0.        ],
       [-1.3517791 , -2.2090058 , -2.5202247 , ..., -1.577823  ,
        -0.68453092,  0.        ]])

In [4]:
y = np.zeros(x.shape[0])
y

array([0., 0., 0., ..., 0., 0., 0.])

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=40)

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [8]:
input_dim = x_train.shape[1]
embedding_dim = 16

In [30]:
encoder = Sequential([
    InputLayer(input_dim),
    Dense(32, "relu"),
    Dense(embedding_dim, "relu")
])

In [10]:
decoder = Sequential([
    InputLayer(embedding_dim),
    Dense(32, "relu"),
    Dense(input_dim, "sigmoid")
])

In [11]:
autoencoder = Sequential([encoder, decoder])

In [12]:
from tensorflow.keras.optimizers import Adam

In [13]:
autoencoder.compile("adam", "mean_squared_error")

In [14]:
autoencoder.fit(x_train, y_train, 32, 10, validation_data=(x_test, y_test))

Epoch 1/10
118/118 [==============================] - 1s 3ms/step - loss: 0.0684 - val_loss: 2.0257e-04
Epoch 2/10
118/118 [==============================] - 0s 2ms/step - loss: 1.1062e-04 - val_loss: 6.0771e-05
Epoch 3/10
118/118 [==============================] - 0s 2ms/step - loss: 4.0187e-05 - val_loss: 2.8453e-05
Epoch 4/10
118/118 [==============================] - 0s 2ms/step - loss: 1.9915e-05 - val_loss: 1.6314e-05
Epoch 5/10
118/118 [==============================] - 0s 2ms/step - loss: 1.1494e-05 - val_loss: 1.0174e-05
Epoch 6/10
118/118 [==============================] - 0s 2ms/step - loss: 6.8988e-06 - val_loss: 6.8398e-06
Epoch 7/10
118/118 [==============================] - 0s 2ms/step - loss: 4.6278e-06 - val_loss: 4.9095e-06
Epoch 8/10
118/118 [==============================] - 0s 2ms/step - loss: 3.3105e-06 - val_loss: 3.6754e-06
Epoch 9/10
118/118 [==============================] - 0s 2ms/step - loss: 2.4613e-06 - val_loss: 2.8784e-06
Epoch 10/10
118/118 [===========

In [15]:
autoencoder.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 16)                5072      
                                                                 
 sequential_1 (Sequential)   (None, 141)               5197      
                                                                 
Total params: 10269 (40.11 KB)
Trainable params: 10269 (40.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
predictions = autoencoder.predict(x_test)
mse = np.mean((x_test - predictions) * 2, axis=1)
mse

40/40 [==============================] - 0s 1ms/step


array([ 0.62064205,  0.14980479, -0.15823695, ...,  0.29968936,
        0.14332785, -0.47080659])

In [23]:
threshold = np.percentile(mse, 95)
y_test_pred = np.where(mse > threshold, 1, 0)
y_test_pred

array([1, 0, 0, ..., 0, 0, 0])

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test_pred)

0.9496